In [1]:
import pandas 
import selenium
import numpy 
import seaborn
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select 


In [20]:
#Read in Config txt file as dict
with open(r"C:\Users\drpow\Documents\Personal Projects\positively_climbable\config.txt", "r") as config_txt:
    config_txt = eval(config_txt.read())
config = config_txt

#mapping rock route type to HTML id + map other settings to vars 
route_type = config["route_type"].lower()
diff_rating_range = config["diff_rating_range"]

rock_type_dict = {"sport":"check_is_sport_climb", "trad":"check_is_trad_climb", "toprope" : "check_is_top_rope"}
rock_type = config["rock_route_type_exclude"]

In [16]:
for i in rock_type:
    print(i)

sport
trad
toprope


In [21]:
driver = webdriver.Chrome(r"C:\Users\drpow\Documents\Personal Projects\positively_climbable\chromedriver.exe")
driver.get("http://www.mountainproject.com")

# Settings 

In [22]:
#apply config settings to "route finder"
select_type = Select(driver.find_element_by_id("type")) 
select_type.select_by_value(route_type)

#lb = lower bound, ub = upper bound
if route_type =='rock': 
    Select(driver.find_element_by_id("diffMinrock")).select_by_visible_text(diff_rating_range[0])
    Select(driver.find_element_by_id("diffMaxrock")).select_by_visible_text(diff_rating_range[1])
    for rock_toggle in rock_type: 
        if rock_type[rock_toggle] == True:
            driver.find_element_by_id(rock_type_dict[rock_toggle]).click()